In [ ]:
import pandas as pd
import numpy as np
import warnings
import re

warnings.filterwarnings('ignore')

# ==============================================================================
# KONFIGURASI FILE INPUT
# ==============================================================================
# Pastikan nama file ini sesuai dengan yang ada di folder Anda
FILE_BBM = "BBM AAB.xlsx"
FILE_HAULAGE = "HAULAGE OKT-NOV 2025.xlsx"
FILE_DOORING_JARAK = "DOORING_WITH_DISTANCE.xlsx" # File hasil dari script jarakDooring.py nanti

# Mapping Nama Unit agar seragam
def clean_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    return re.sub(r'[^A-Z0-9]', '', name)

# ==============================================================================
# FUNGSI MAPPING JARAK HAULAGE (Updated 26-Feb-2026)
# ==============================================================================
def get_haulage_distance(row):
    # Bersihkan input agar tidak sensitif spasi/huruf besar-kecil
    rute = str(row['RUTE']).strip().upper()
    depo = str(row['DEPO']).strip().upper() if pd.notna(row['DEPO']) else ""
    jenis_cont = str(row['JENIS CONT']).strip().upper() if pd.notna(row['JENIS CONT']) else ""

    # --- 1. MAPPING RUTE SPESIFIK & CONDITIONAL ---
    
    # RUTE: BERLIAN - DEPO (Cek Kolom DEPO)
    if rute == "BERLIAN - DEPO":
        if "UDATIN" in depo: return 0.5      # 500 m
        if "MIRAH" in depo: return 1.1       # 1.1 km
        if "UNGGUL" in depo: return 0.75     # 750 m
        if "I POWER" in depo: return 0.9     # 900 m
        if "MARUMAN" in depo: return 0.27    # 270 m
        if "EXIM" in rute: return 2.775      # Fallback jika tertulis di rute
        return 2.775 # Default BERLIAN - DEPO

    # RUTE: DEPO - BERLIAN (Cek Kolom DEPO)
    if rute == "DEPO - BERLIAN":
        if "UDATIN" in depo: return 0.85     # 850 m
        if "MIRAH" in depo: return 2.8
        if "UNGGUL" in depo: return 2.2
        if "I POWER" in depo: return 0.65    # 650 m
        return 2.575 # Default DEPO - BERLIAN

    # RUTE: DEPO (Nama Depo Langsung di Rute) - BERLIAN
    if "DEPO (UDATIN) - BERLIAN" in rute: return 0.85
    if "DEPO (MIRAH) - BERLIAN" in rute: return 2.8
    if "DEPO (UNGGUL) - BERLIAN" in rute: return 2.2
    if "DEPO (I POWER) - BERLIAN" in rute: return 0.65

    # RUTE: BERLIAN - DEPO (EXIM) & DEPO - BERLIAN (EXIM)
    if "BERLIAN - DEPO (EXIM)" in rute: return 2.775
    if "DEPO - BERLIAN (EXIM)" in rute: return 2.575

    # RUTE: DEPO - KALIANAK (Cek Jenis Cont)
    if rute == "DEPO - KALIANAK" or "DEPO - KALIANAK" in rute:
        if "GENSET" in jenis_cont: return 5.15
        if "20FT" in jenis_cont or "40FT" in jenis_cont: return 9.35
        return 9.35 # Default

    # RUTE: KALIANAK - DEPO (Cek Jenis Cont)
    if rute == "KALIANAK - DEPO" or "KALIANAK - DEPO" in rute:
        if "GENSET" in jenis_cont: return 4.975
        if "20FT" in jenis_cont or "40FT" in jenis_cont: return 8.975
        return 8.975 # Default

    # RUTE: KALIANAK - BERLIAN
    if "KALIANAK - BERLIAN" in rute:
        if "GENSET" in jenis_cont: return 6.4
        return 6.4

    # RUTE: BERLIAN - KALIANAK (EXIM)
    if "BERLIAN - KALIANAK" in rute: return 11.0
    
    # RUTE: KALIANAK - TELUK LAMONG / TTL
    if "KALIANAK - TELUK LAMONG" in rute or "KALIANAK - TERMINAL TELUK LAMONG" in rute: return 9.2

    # RUTE: KALIANAK - TAMBAK LANGON
    if "KALIANAK - TAMBAK LANGON" in rute or "KALIANAK-TAMBAK LANGON" in rute: return 2.7

    # RUTE: NILAM - KALIANAK
    if "NILAM - KALIANAK" in rute: return 10.3

    # RUTE: TELUK LAMONG - KALIANAK
    if "TELUK LAMONG - KALIANAK" in rute: return 8.6

    # RUTE: TAMBAK LANGON - KALIANAK
    if "TAMBAK LANGON - KALIANAK" in rute: return 3.3

    # --- 2. MAPPING RUTE LANGSUNG (EXACT MATCH) ---
    exact_map = {
        "BERLIAN - TAMBAK LANGON": 11.5,
        "BERLIAN - TERMINAL TELUK LAMONG": 18.1,
        "BERLIAN - TTL": 18.1,
        "DEPO - DEPO": 2.825,
        "DEPO - ICT": 7.65,
        "ICT - DEPO": 4.55,
        "DEPO - KALOG": 2.275,
        "DEPO - NILAM": 2.375,
        "DEPO PERAK - TAMBAK LANGON": 10.025,
        "DEPO PERAK - TELUK LAMONG": 16.6,
        "ICT/TPS - TAMBAK LANGON": 13.2,
        "ICT/TPS - TELUK LAMONG": 19.8,
        "JAMRUD - DEPO": 5.6,
        "KADE - KADE": 2.0, # Asumsi sementara
        "KALIANAK - BERLIAN (JENIS CONT GENSET)": 6.4,
        "KALIANAK - DEPO (JENIS CONT 20FT, 40FT)": 10.4, # Koreksi spesifik
        "KALIANAK - TELUK LAMONG (JENIS CONT 20FT, 40FT)": 9.2,
        "KALIANAK-TAMBAK LANGON (JENIS CONT 20FT, 40FT)": 2.7,
        "KALOG - BERLIAN": 3.4,
        "KALOG - DEPO": 3.575,
        "KALOG - NILAM": 3.7,
        "KALOG - TAMBAK LANGON": 9.4,
        "KALOG - TELUK LAMONG": 16.0,
        "MIRAH - DEPO": 4.975,
        "NILAM - DEPO": 2.325,
        "NILAM - KALIANAK (JENIS CONT 20FT, 40FT)": 10.3,
        "NILAM - TAMBAK LANGON": 10.8,
        "NILAM - TERMINAL TELUK LAMONG": 17.4,
        "TAMBAK LANGON - BERLIAN": 10.2,
        "TAMBAK LANGON - DEPO PERAK": 8.875,
        "TAMBAK LANGON - KALIANAK (JENIS CONT 20FT, 40FT)": 3.3,
        "TAMBAK LANGON - NILAM": 10.5,
        "TAMBAK LANGON - TTL": 6.7,
        "TAMBAK LANGON - TERMINAL TELUK LAMONG": 6.7,
        "TTL - TAMBAK LANGON": 6.4,
        "TERMINAL TELUK LAMONG - TAMBAK LANGON": 6.4,
        "TELUK LAMONG - DEPO PERAK": 14.325,
        "TELUK LAMONG - ICT/TPS": 20.6,
        "TELUK LAMONG - KALIANAK (JENIS CONT 20FT, 40FT)": 8.6,
        "TELUK LAMONG - KALOG": 14.3,
        "TERMINAL TELUK LAMONG - BERLIAN": 15.5,
        "TERMINAL TELUK LAMONG - NILAM": 15.9,
        "OPER SHIFT BERLIAN-DEPO": 2.775,
        "OPER SHIFT DEPO - BERLIAN": 2.575,
        "OPER SHIFT JAMRUD - DEPO": 5.6,
        "OPER SHIFT NILAM-DEPO": 2.325
    }

    if rute in exact_map:
        return exact_map[rute]
        
    # Fallback cek partial string untuk rute yang mungkin sedikit beda penulisan
    if "BERLIAN" in rute and "UDATIN" in rute: return 0.5
    if "BERLIAN" in rute and "MIRAH" in rute: return 1.1
    
    return 0 # Jika tidak ditemukan

In [ ]:
# ==============================================================================
# PROSES DATA BBM (SUM LITER PER UNIT)
# ==============================================================================
print("1. Memproses Data BBM...")
xls_bbm = pd.ExcelFile(FILE_BBM)
all_bbm_data = []

sheets_bbm = {
    'OKT': 'Oktober', 'NOV': 'November' 
}

for sheet_name, bulan in sheets_bbm.items():
    if sheet_name in xls_bbm.sheet_names:
        df_sheet = pd.read_excel(xls_bbm, sheet_name=sheet_name, header=None)
        
        # Cari baris header yang mengandung 'LITER'
        header_row_idx = None
        for idx, row in df_sheet.iterrows():
            if row.astype(str).str.contains('LITER', case=False).any():
                header_row_idx = idx
                break
        
        if header_row_idx is not None:
            # Baris di atas header biasanya nama unit
            unit_names = df_sheet.iloc[header_row_idx - 2].ffill() 
            headers = df_sheet.iloc[header_row_idx]
            data_rows = df_sheet.iloc[header_row_idx + 1:]
            
            for col_idx in range(len(headers)):
                header_text = str(headers.iloc[col_idx]).upper().strip()
                if header_text == 'LITER':
                    unit_raw = str(unit_names.iloc[col_idx])
                    if pd.notna(unit_raw) and unit_raw.strip() != '':
                        liter_vals = pd.to_numeric(data_rows.iloc[:, col_idx], errors='coerce').sum()
                        if liter_vals > 0:
                            all_bbm_data.append({
                                'EQUIP NAME': unit_raw,
                                'LITER': liter_vals
                            })

df_bbm_total = pd.DataFrame(all_bbm_data).groupby('EQUIP NAME')['LITER'].sum().reset_index()
df_bbm_total['Unit_Clean'] = df_bbm_total['EQUIP NAME'].apply(clean_unit_name)
print(f"   -> Total Unit BBM ditemukan: {len(df_bbm_total)}")


# ==============================================================================
# PROSES HAULAGE
# ==============================================================================
print("2. Memproses Data Haulage...")
try:
    df_haulage = pd.read_excel(FILE_HAULAGE)
    
    # Hitung Jarak per baris
    df_haulage['Jarak_Km'] = df_haulage.apply(get_haulage_distance, axis=1)
    
    # Asumsi kolom berat bernama 'BERAT' atau 'WEIGHT'. Sesuaikan jika beda.
    # Jika tidak ada kolom berat, kita anggap 0 dulu (nanti dicek user)
    col_berat_haulage = next((c for c in df_haulage.columns if 'BERAT' in c.upper() or 'WEIGHT' in c.upper()), None)
    
    if col_berat_haulage:
        df_haulage['Berat_Ton'] = pd.to_numeric(df_haulage[col_berat_haulage], errors='coerce').fillna(0) / 1000 # Asumsi input Kg
    else:
        df_haulage['Berat_Ton'] = 0 
        print("   [Warning] Kolom Berat tidak ditemukan di file Haulage.")

    df_haulage['TonKm'] = df_haulage['Berat_Ton'] * df_haulage['Jarak_Km']
    
    # Agregat per Unit
    col_unit_haulage = next((c for c in df_haulage.columns if 'NOPOL' in c.upper() or 'UNIT' in c.upper()), 'NOPOL')
    df_haulage['Unit_Clean'] = df_haulage[col_unit_haulage].apply(clean_unit_name)
    
    df_haulage_group = df_haulage.groupby('Unit_Clean').agg({
        'TonKm': 'sum',
        'Jarak_Km': 'sum', # Total KM tempuh
        'Berat_Ton': 'sum'
    }).reset_index().rename(columns={'TonKm': 'TonKm_Haulage', 'Jarak_Km': 'Km_Haulage', 'Berat_Ton': 'Ton_Haulage'})
    
    print(f"   -> Data Haulage diproses. Total Unit: {len(df_haulage_group)}")

except Exception as e:
    print(f"   [Error] Gagal memproses Haulage: {e}")
    df_haulage_group = pd.DataFrame(columns=['Unit_Clean', 'TonKm_Haulage', 'Km_Haulage', 'Ton_Haulage'])

In [ ]:

# ==============================================================================
# PROSES DOORING (Placeholder / Menunggu File Hasil Script Python)
# ==============================================================================
print("3. Memproses Data Dooring...")
try:
    # Cek apakah file dooring hasil python sudah ada
    import os
    if os.path.exists(FILE_DOORING_JARAK):
        df_dooring = pd.read_excel(FILE_DOORING_JARAK)
        # Pastikan kolom TonKm_Dooring sudah dihitung di script Python
        # Jika belum, hitung disini: df_dooring['TonKm_Dooring'] = df_dooring['Berat'] * df_dooring['Jarak_PP_Km']
        
        col_unit_dooring = next((c for c in df_dooring.columns if 'NOPOL' in c.upper() or 'UNIT' in c.upper()), 'NOPOL')
        df_dooring['Unit_Clean'] = df_dooring[col_unit_dooring].apply(clean_unit_name)
        
        df_dooring_group = df_dooring.groupby('Unit_Clean').agg({
            'TonKm_Dooring': 'sum',
            'Jarak_PP_Km': 'sum',
            'Berat_Total_Ton': 'sum' # Sesuaikan nama kolom berat di file hasil dooring
        }).reset_index().rename(columns={'Jarak_PP_Km': 'Km_Dooring', 'Berat_Total_Ton': 'Ton_Dooring'})
        
        print(f"   -> Data Dooring ditemukan. Total Unit: {len(df_dooring_group)}")
    else:
        print("   [Info] File Dooring (dengan jarak) belum tersedia. Skip Dooring.")
        df_dooring_group = pd.DataFrame(columns=['Unit_Clean', 'TonKm_Dooring', 'Km_Dooring', 'Ton_Dooring'])

except Exception as e:
    print(f"   [Error] Gagal memproses Dooring: {e}")
    df_dooring_group = pd.DataFrame(columns=['Unit_Clean', 'TonKm_Dooring', 'Km_Dooring', 'Ton_Dooring'])


# ==============================================================================
# GABUNGKAN SEMUA (MERGE) & EXPORT
# ==============================================================================
print("4. Menggabungkan Data...")

# Gabung Haulage & Dooring (Outer Join karena unit bisa saja cuma haulage atau cuma dooring)
df_ops = pd.merge(df_haulage_group, df_dooring_group, on='Unit_Clean', how='outer').fillna(0)

# Total TonKm & Total Km
df_ops['Total_TonKm'] = df_ops['TonKm_Haulage'] + df_ops['TonKm_Dooring']
df_ops['Total_Km'] = df_ops['Km_Haulage'] + df_ops['Km_Dooring']
df_ops['Total_Ton_Angkut'] = df_ops['Ton_Haulage'] + df_ops['Ton_Dooring']

# Gabung dengan Data BBM (Inner Join - hanya unit yang ada BBM-nya yang dianalisa)
df_final = pd.merge(df_ops, df_bbm_total, on='Unit_Clean', how='inner')

# Hitung Ratio
df_final['L_per_TonKm'] = np.where(df_final['Total_TonKm'] > 0, df_final['LITER'] / df_final['Total_TonKm'], 0)
df_final['Km_per_L'] = np.where(df_final['LITER'] > 0, df_final['Total_Km'] / df_final['LITER'], 0)

# Export
output_file = "HASIL_ANALISA_TRUCKING_OKT_NOV.xlsx"
df_final.to_excel(output_file, index=False)
print(f"SELESAI! File output: {output_file}")
print(df_final.head())